In [133]:
import pandas as pd
import numpy as np
from docplex.cp.model import *

ModuleNotFoundError: No module named 'docplex'

In [137]:
df_name1 = pd.read_excel('../data/생관3-612-24-004_2024년 3월 일일생산계획 및 업체별 차종현황_Rev.01_24.03.06(확정).xlsx', skiprows=[0, 1, 2], usecols='C', nrows=64)
df_name2 = pd.read_excel('../data/생관3-612-24-007_2024년 4월 일일생산계획 및 업체별 차종현황_Rev.00_24.03.28(확정).xlsx', skiprows=[0, 1, 2], usecols='C', nrows=64)

In [138]:
set(df_name1) == set(df_name2)

True

In [41]:
df_name = pd.read_excel('../data/생관3-612-24-004_2024년 3월 일일생산계획 및 업체별 차종현황_Rev.01_24.03.06(확정).xlsx', skiprows=[0, 1, 2], usecols='C', nrows=64)
df_data = pd.read_excel('../data/생관3-612-24-004_2024년 3월 일일생산계획 및 업체별 차종현황_Rev.01_24.03.06(확정).xlsx', skiprows=[0, 1, 2, 3], usecols='F:BE', nrows=64)
df_data2 = pd.read_excel('../data/생관3-612-24-007_2024년 4월 일일생산계획 및 업체별 차종현황_Rev.00_24.03.28(확정).xlsx', skiprows=[0, 1, 2, 3], usecols='F:BE', nrows=64)

In [42]:
df_name = df_name.drop(index=df_name.index[0])
df_name = df_name.reset_index(drop=True)
df_name = df_name.map(lambda x: x.strip() if isinstance(x, str) else x)
df_name = df_name.ffill()
df_data = df_data.drop(index=df_data.index[-1])
df_data2 = df_data2.drop(index=df_data2.index[-1])
df_data.index = df_name['세부공정명'].tolist()
df_data2.index = df_name['세부공정명'].tolist()

In [43]:
def clean_column_names(df):
    columns = df.columns.tolist()
    for i in range(1, len(columns)):  # 첫 번째 열은 처리할 필요 없음
        if isinstance(columns[i], str) and "Unnamed" in columns[i]:
            columns[i] = columns[i - 1]  # 바로 왼쪽 열 이름으로 대체
    df.columns = columns
    return df

# DataFrame 열 이름 정리
df_data = clean_column_names(df_data)
df_data2 = clean_column_names(df_data2)

In [44]:
class SubwayCar:
    def __init__(self, name):
        self.name = name
        self.activity_dict = dict()
    
    def add_activity(self, activity_name, date):
        if activity_name not in self.activity_dict:
            self.activity_dict[activity_name] = date
        else:
            print(f"subwaycar {self.name} activity {activity_name}: {date} is already taken")

In [45]:
def create_subway_car_dict(df_data, subway_car_dict, start_date):
    for col_idx in range(df_data.shape[1]):
            date = df_data.columns[col_idx]  # 현재 열의 이름
    
            for row_idx in range(df_data.shape[0]):  # 행 반복
                activity_name = df_data.index[row_idx]  # 현재 행 이름
                car_name = df_data.iloc[row_idx, col_idx]  # 값 가져오기
    
                if pd.notna(car_name):  # NaN 값 제외
                    if car_name not in subway_car_dict:
                        subway_car_dict[car_name] = SubwayCar(name=car_name)
                    
                    subway_car_dict[car_name].add_activity(activity_name, start_date + pd.Timedelta(days=date - 1))
    return subway_car_dict

In [46]:
subway_car_dict1 = {}  # {name: SubwayCar 객체}

subway_car_dict1 = create_subway_car_dict(df_data, subway_car_dict1, pd.to_datetime('2024-03-01'))
print('----------------')
print('----------------')
print('----------------')
print('----------------')
print('----------------')
subway_car_dict2 = {}  # {name: SubwayCar 객체}

subway_car_dict2 = create_subway_car_dict(df_data2, subway_car_dict2, pd.to_datetime('2024-04-01'))



----------------
----------------
----------------
----------------
----------------


In [47]:
# 결과 확인
for car_name, subway_car in subway_car_dict1.items():
    print(f"SubwayCar Name: {car_name}")
    print("Activities:", subway_car.activity_dict)

SubwayCar Name: S025-M2
Activities: {'측창취부/T-BOLT 삽입': Timestamp('2024-03-01 00:00:00'), '실내/상하 CABLE HARNESS 취부': Timestamp('2024-03-04 00:00:00'), '실내/상하 배선': Timestamp('2024-03-06 00:00:00'), '실내 배선 작업': Timestamp('2024-03-07 00:00:00'), '하부덕트검사\n (D+1~4일차 조정작업)': Timestamp('2024-03-08 00:00:00'), '실내 배선 검사': Timestamp('2024-03-09 00:00:00')}
SubwayCar Name: S018-M1
Activities: {'실내/상하 배선': Timestamp('2024-03-01 00:00:00'), '실내 배선 작업': Timestamp('2024-03-02 00:00:00'), '하부덕트검사\n (D+1~4일차 조정작업)': Timestamp('2024-03-04 00:00:00'), '객실도어 취부': Timestamp('2024-03-11 00:00:00'), '객실도어 취부 검사': Timestamp('2024-03-12 00:00:00'), '배관 MODULE 취부': Timestamp('2024-03-12 00:00:00'), '제동기기 취부 및 누설검사': Timestamp('2024-03-14 00:00:00'), 'COUPLER 취부': Timestamp('2024-03-15 00:00:00'), 'D+5~13일차 조정작업': Timestamp('2024-03-15 00:00:00'), '상하 전장기기 취부 결선': Timestamp('2024-03-16 00:00:00'), 'AIR DUCT MODULE 취부': Timestamp('2024-03-19 00:00:00'), 'CENTER GRILL 취부': Timestamp('2024-03-20 00:00:00'), 'CENTER 

In [48]:
for car_name, subway_car in subway_car_dict2.items():
    print(f"SubwayCar Name: {car_name}")
    print("Activities:", subway_car.activity_dict)

SubwayCar Name: S036-T2
Activities: {'흡음재 취부': Timestamp('2024-04-01 00:00:00'), '흡음재 씰링/검사/수정': Timestamp('2024-04-02 00:00:00'), '측창취부/T-BOLT 삽입': Timestamp('2024-04-02 00:00:00'), '리무벌/파티션 프레임 취부': Timestamp('2024-04-03 00:00:00'), '실내/상하 CABLE HARNESS 취부': Timestamp('2024-04-04 00:00:00'), '실내/상하 배선': Timestamp('2024-04-05 00:00:00'), '객실도어 취부': Timestamp('2024-04-06 00:00:00'), '하부덕트검사\n (D+1~4일차 조정작업)': Timestamp('2024-04-08 00:00:00'), '실내 배선 검사': Timestamp('2024-04-09 00:00:00'), 'AIR DUCT MODULE 취부': Timestamp('2024-04-27 00:00:00'), 'CENTER GRILL 취부': Timestamp('2024-04-29 00:00:00'), 'AIR DUCT MODULE 취부 검사': Timestamp('2024-04-30 00:00:00')}
SubwayCar Name: S035-M2
Activities: {'실내/상하 CABLE HARNESS 취부': Timestamp('2024-04-01 00:00:00'), '실내/상하 배선': Timestamp('2024-04-02 00:00:00'), '객실도어 취부': Timestamp('2024-04-03 00:00:00'), '하부덕트검사\n (D+1~4일차 조정작업)': Timestamp('2024-04-04 00:00:00'), '실내 배선 검사': Timestamp('2024-04-05 00:00:00'), 'AIR DUCT MODULE 취부': Timestamp('2024-04-26 

In [134]:
process_to_part = {
    "하부": ['흡음재 취부', '흡음재 씰링/검사/수정', '측창취부/T-BOLT 삽입', '실내/상하 CABLE HARNESS 취부', '실내/상하 배선', '실내 배선 작업', '객실도어 취부', '객실도어 취부 검사', '하부덕트검사\n (D+1~4일차 조정작업)', 'CAB MODULE 취부(TC)', '배전반 취부 및 배선작업', '운전실 내장판 취부', '운전실 캐비넷 취부(TC)', '운전실 배전반 결선(TC)', '운전실 DOOR 취부(TC)', '도통검사(량단위 시험기)', '도통 자체 검사(완료)', '도통 검사 수정', '도통입회검사/내전압자체검사', '내전압 입회검사', '수정작업 및 점검커버 복구', 'D+17~20일차 조정작업', '전장취부, 결선 복구 완료', '실내 조정 및 실내·외 설비 완료', '전장품 취부 입회검사', '복구 및 수정작업', '실내 설비 입회검사', '대차차입 전검사 / 대차차입', '대차 전장품 취부 결선 & 마무리', '대차차입 후 검사'],
    "실내": ['리무벌/파티션 프레임 취부', '실내 배선 검사', 'AIR DUCT MODULE 취부', 'CENTER GRILL 취부', 'AIR DUCT MODULE 취부 검사', '배관 MODULE 취부', 'CENTER PIVOT 취부', 'COUPLER 취부', '제동기기 취부 및 누설검사', 'D+5~13일차 조정작업', 'ROOF 내장판 취부', 'SIDE 내장판 취부', 'AIR CON 취부', '내장판 취부 검사', '전장기기 취부 결선 완료', '운전실 전장기기 취부 결선(TC)'],
    "옥상": ['상하 전장기기 취부 결선', '내장판 조정작업(완료)']
}


In [135]:
temp = 0
for part, process in process_to_part.items():
    temp += len(process)
print(temp)

48


In [136]:
for part in ['하부', '실내', '옥상']:
    for car_name, subway_car in subway_car_dict2.items():
        temp_list = []
        for process, date in subway_car.activity_dict.items():
            if process in process_to_part[part]:
                temp_list.append((process_to_part[part].index(process), date))
                
        for i in range(len(temp_list) - 1):
            for j in range(i + 1, len(temp_list)):
                index_i, date_i = temp_list[i]
                index_j, date_j = temp_list[j]
                
                # 조건: index가 더 작은데 날짜가 더 크거나, index가 큰데 날짜가 작은 경우
                if (index_i < index_j and date_i > date_j) or (index_i > index_j and date_i < date_j):
                    print(f"Error in {car_name}, Part: {part}")
                    print(f"{process_to_part[part][index_i]} // {process_to_part[part][index_j]}")
                    print(f"Dates: {date_i} and {date_j}")


Error in S040-TC2, Part: 하부
객실도어 취부 // 실내 배선 작업
Dates: 2024-04-01 00:00:00 and 2024-04-29 00:00:00


In [119]:
process_list = [
    "흡음재 취부",
    "흡음재 씰링/검사/수정",
    "측창취부/T-BOLT 삽입",
    "실내/상하 CABLE HARNESS 취부",
    "실내/상하 배선",
    "실내 배선 작업",
    "객실도어 취부",
    "리무벌/파티션 프레임 취부",
    "객실도어 취부 검사",
    "하부덕트검사\n (D+1~4일차 조정작업)",
    "실내 배선 검사",
    "AIR DUCT MODULE 취부",
    "CENTER GRILL 취부",
    "AIR DUCT MODULE 취부 검사",
    "CAB MODULE 취부(TC)",
    "배관 MODULE 취부",
    "배전반 취부 및 배선작업",
    "CENTER PIVOT 취부",
    "COUPLER 취부",
    "제동기기 취부 및 누설검사",
    "운전실 내장판 취부",
    "D+5~13일차 조정작업",
    "ROOF 내장판 취부",
    "운전실 캐비넷 취부(TC)",
    "상하 전장기기 취부 결선",
    "SIDE 내장판 취부",
    "운전실 배전반 결선(TC)",
    "AIR CON 취부",
    "내장판 조정작업(완료)",
    "내장판 취부 검사",
    "운전실 DOOR 취부(TC)",
    "전장기기 취부 결선 완료",
    "운전실 전장기기 취부 결선(TC)",
    "도통검사(량단위 시험기)",
    "도통 자체 검사(완료)",
    "도통 검사 수정",
    "도통입회검사/내전압자체검사",
    "내전압 입회검사",
    "수정작업 및 점검커버 복구",
    "D+17~20일차 조정작업",
    "전장취부, 결선 복구 완료",
    "실내 조정 및 실내·외 설비 완료",
    "전장품 취부 입회검사",
    "복구 및 수정작업",
    "실내 설비 입회검사",
    "대차차입 전검사 / 대차차입",
    "대차 전장품 취부 결선 & 마무리",
    "대차차입 후 검사"
]
print(len(process_list))

48
